<a href="https://colab.research.google.com/github/zainabbio/Data_Science/blob/main/HLA_DR_ML_CRSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import scipy.io as sio
import numpy as np
from sklearn.preprocessing import scale
from catboost import CatBoostClassifier
import lightgbm as lgb

import os

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
train_esm1b_P = pd.read_csv('/content/HLA_main_pos_CPCR.csv')

train_esm1b_N = pd.read_csv('/content/HLA_main_neg_CPCR.csv')

train_esm1b = np.row_stack((train_esm1b_P, train_esm1b_N))

In [ ]:
[m1, n1] = np.shape(train_esm1b)
label1 = np.ones((20175, 1))
label2 = np.zeros((20385, 1))
label = np.append(label1, label2)

# Standardized dataset
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_esm1b)

shu = scale(train_esm1b)

#shu = scale(train_esm1b)
#X = np.reshape(shu, (-1, 1, n1))
X = shu
y = label

# Divide the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(32448, 71)
(8112, 71)
(32448,)
(8112,)


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

In [ ]:
parameters = {'depth'         : [4,5,6,7,8,9, 10],
                 'learning_rate' : [0.01,0.02,0.03,0.04],
                  'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
                 }

In [ ]:
model  = CatBoostClassifier().fit(X_train, y_train)

Learning rate set to 0.045521
0:	learn: 0.6893087	total: 62.2ms	remaining: 1m 2s
1:	learn: 0.6849236	total: 77.9ms	remaining: 38.9s
2:	learn: 0.6811345	total: 97.3ms	remaining: 32.3s
3:	learn: 0.6775392	total: 113ms	remaining: 28s
4:	learn: 0.6742427	total: 128ms	remaining: 25.5s
5:	learn: 0.6709721	total: 145ms	remaining: 24.1s
6:	learn: 0.6681315	total: 161ms	remaining: 22.9s
7:	learn: 0.6656426	total: 177ms	remaining: 22s
8:	learn: 0.6632677	total: 195ms	remaining: 21.5s
9:	learn: 0.6610010	total: 210ms	remaining: 20.8s
10:	learn: 0.6591397	total: 225ms	remaining: 20.2s
11:	learn: 0.6571135	total: 240ms	remaining: 19.7s
12:	learn: 0.6550973	total: 256ms	remaining: 19.4s
13:	learn: 0.6531886	total: 295ms	remaining: 20.8s
14:	learn: 0.6515404	total: 350ms	remaining: 23s
15:	learn: 0.6499498	total: 399ms	remaining: 24.6s
16:	learn: 0.6482657	total: 452ms	remaining: 26.2s
17:	learn: 0.6468750	total: 502ms	remaining: 27.4s
18:	learn: 0.6450343	total: 553ms	remaining: 28.6s
19:	learn: 0.6

In [ ]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [ ]:
def confusion_matrix_scorer(clf, X_train, y_train):
        y_pred = clf.predict(X_train)
        cm = confusion_matrix(y_train, y_pred)
        return {'tn': cm[0, 0], 'fp': cm[0, 1],
                'fn': cm[1, 0], 'tp': cm[1, 1]}

In [ ]:
cv_results = cross_validate(model, X_train, y_train, cv=5,
                            scoring=confusion_matrix_scorer)

# Getting the test set true positive scores
TP = cv_results['test_tp'].mean()

# Getting the test set false negative scores
FN = cv_results['test_fn'].mean()

# Getting the test set false positive scores
FP = cv_results['test_fp'].mean()

# Getting the test set true negative scores
TN = cv_results['test_tn'].mean()

Streaming output truncated to the last 5000 lines.
4:	learn: 0.6763949	total: 246ms	remaining: 48.9s
5:	learn: 0.6734769	total: 288ms	remaining: 47.7s
6:	learn: 0.6708258	total: 326ms	remaining: 46.3s
7:	learn: 0.6684405	total: 366ms	remaining: 45.4s
8:	learn: 0.6657078	total: 406ms	remaining: 44.7s
9:	learn: 0.6635832	total: 458ms	remaining: 45.4s
10:	learn: 0.6617383	total: 491ms	remaining: 44.2s
11:	learn: 0.6598157	total: 527ms	remaining: 43.4s
12:	learn: 0.6578466	total: 574ms	remaining: 43.6s
13:	learn: 0.6559138	total: 608ms	remaining: 42.8s
14:	learn: 0.6543014	total: 627ms	remaining: 41.2s
15:	learn: 0.6528227	total: 644ms	remaining: 39.6s
16:	learn: 0.6512607	total: 658ms	remaining: 38.1s
17:	learn: 0.6498587	total: 685ms	remaining: 37.3s
18:	learn: 0.6482639	total: 720ms	remaining: 37.2s
19:	learn: 0.6468799	total: 752ms	remaining: 36.9s
20:	learn: 0.6451391	total: 786ms	remaining: 36.6s
21:	learn: 0.6437966	total: 822ms	remaining: 36.5s
22:	learn: 0.6428073	total: 853ms	rem

In [ ]:
####TRAINING###
acurracy = (TP+TN) / (TP+TN+FP+FN)
F1_score = 2*TP / ((2*TP) + (FP + FN))
precision = TP / (TP + FP)
specificity = TN / (FP + TN)
sensitivity_recall = TP / (TP + FN)
import math
MCC = ((TP*TN) - (FP*FN)) / math.sqrt(((TP+FP)*(TP+FN))*((TN+FP)*(TN+FN)))

print("Accuracy: ", acurracy)
print("F1_score: ", F1_score)
print("Precision: ", precision)
print("Specificity: ", specificity)
print("Sensitivity/Recall: ", sensitivity_recall)
print("MCC: ", MCC)

Accuracy:  0.7925912228796844
F1_score:  0.7924633033181202
Precision:  0.7907076923076923
Specificity:  0.7909649661954518
Sensitivity/Recall:  0.7942267276548399
MCC:  0.5851900931261915


In [ ]:
####TESTING###
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
pred_test=model.predict(X_test)

conf = confusion_matrix(y_test, pred_test)
TP = conf[1, 1]
FP = conf[0, 1]
TN = conf[0, 0]
FN = conf[1, 0]

acurracy = (TP+TN) / (TP+TN+FP+FN)
F1_score = 2*TP / ((2*TP) + (FP + FN))
precision = TP / (TP + FP)
specificity = TN / (FP + TN)
sensitivity_recall = TP / (TP + FN)

import math
MCC = ((TP*TN) - (FP*FN)) / math.sqrt(((TP+FP)*(TP+FN))*((TN+FP)*(TN+FN)))

print("Accuracy: ", acurracy)
print("F1_score: ", F1_score)
print("Precision: ", precision)
print("Specificity: ", specificity)
print("Sensitivity/Recall: ", sensitivity_recall)
print("MCC: ", MCC)

Accuracy:  0.8099112426035503
F1_score:  0.8093000247341083
Precision:  0.8001956468574224
Specificity:  0.8014580801944107
Sensitivity/Recall:  0.8186139604703527
MCC:  0.6200269567132211
